# This notebook will serve for the analysis of the Pictures and Statistics for the TFW project

In [2]:
# Import necassary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the datasets
df_statistics = pd.read_csv('../data/statistics_20210707.csv')
df_pictures = pd.read_csv('../data/pictures_dataset.csv')

In [4]:
df_statistics.head(2)

,yearmonth,listing_id,inquiry_count,inquiry_volume,binding_inquiry_count,binding_inquiry_volume,direct_booking_count,direct_booking_volume,result_views,expose_views,phone_clicks
0,2019/07,f1133916-825e-5e2a-93be-eaba28a966e8,2,1352,0,0,0,0,1109,85,1
1,2019/07,836959d3-611c-5edb-887c-273aba5bd102,0,0,0,0,0,0,871,23,2


In [5]:
df_pictures.head(2)

,Unnamed: 0,listing_id,added,activated,description,start_pic_summer,start_pic_winter,picture_resolution
0,0,fbd74bce-23dd-53e7-b988-8beb85160d6f,2009-12-03 00:00:00,2010-12-16 00:00:00,Furnishing,0,0,High-definition
1,1,4e012316-0f53-5fc6-a17f-580ff832554e,2010-01-05 00:00:00,2010-12-16 00:00:00,Floor plan,0,0,Poor Resolution


In [6]:
print(df_pictures.shape)
print(df_statistics.shape)

(644821, 8)
(674604, 11)


In [7]:
df_pictures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644821 entries, 0 to 644820
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          644821 non-null  int64 
 1   listing_id          644821 non-null  object
 2   added               644821 non-null  object
 3   activated           644821 non-null  object
 4   description         644821 non-null  object
 5   start_pic_summer    644821 non-null  int64 
 6   start_pic_winter    644821 non-null  int64 
 7   picture_resolution  644821 non-null  object
dtypes: int64(3), object(5)
memory usage: 39.4+ MB


In [8]:
df_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674604 entries, 0 to 674603
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   yearmonth               674604 non-null  object
 1   listing_id              674604 non-null  object
 2   inquiry_count           674604 non-null  int64 
 3   inquiry_volume          674604 non-null  int64 
 4   binding_inquiry_count   674604 non-null  int64 
 5   binding_inquiry_volume  674604 non-null  int64 
 6   direct_booking_count    674604 non-null  int64 
 7   direct_booking_volume   674604 non-null  int64 
 8   result_views            674604 non-null  int64 
 9   expose_views            674604 non-null  int64 
 10  phone_clicks            674604 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 56.6+ MB


## Remove unnamed column in df pictures

In [10]:
# Drop unnamed column
df_pictures = df_pictures.drop('Unnamed: 0', axis=1)

## Covert date features into datetime

In [9]:
# Convert to datetime
df_statistics['yearmonth'] = pd.to_datetime(df_statistics['yearmonth'])
df_pictures['added'] = pd.to_datetime(df_pictures['added'])
df_pictures['activated'] = pd.to_datetime(df_pictures['activated'])

In [11]:
df_pictures.listing_id.nunique()

30222

In [14]:
df_pictures.activated.nunique()

560004

In [13]:
df_statistics.listing_id.nunique()

30269

In [ ]:
df_statistics.